In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as tfl

2022-12-05 16:46:54.941500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.metrics as metrics

In [3]:

x_train = np.load("./cleaned_data/small/x_train.npy")
y_train = np.load("./cleaned_data/small/y_train.npy")
train_mu = np.load("./cleaned_data/small/train_mu.npy")
train_std = np.load("./cleaned_data/small/train_std.npy")
x_val = np.load("./cleaned_data/small/x_val.npy")
y_val= np.load("./cleaned_data/small/y_val.npy")


In [4]:
# [no sack, sack, time to sack]
y_train[0:2]

array([[ 1.,  0., -1.],
       [ 1.,  0., -1.]])

In [5]:
import keras.backend as K

def my_loss(y_true, y_pred):
    bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    mse = tf.keras.losses.MeanSquaredError()
    return bce(y_true[:,0:-1], y_pred[:,0:-1]) + mse(y_true[:,-1], y_pred[:,-1])

def bce_metric(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true[:,0:-1], y_pred[:,0:-1], from_logits=False))

def mse_metric(y_true, y_pred):
    return K.mean(K.square(y_pred[:,-1] - y_true[:,-1]), axis=-1)

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true[:,1] * y_pred[:,1], 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true[:,1], 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true[:,1] * y_pred[:,1], 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred[:,1], 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


In [6]:
''' 
Model

how cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md
'''

' \nModel\n\nhow cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md\n'

In [7]:
''' 
Output:
[prob of no sack, prob of sack, time till sack]

If predict no sack, doesn't matter what time is (recorded as -1 in training data)
'''

def createModel(input_shape = (23,7)):
    
    X = tfl.Input(input_shape)  # define the input to the model
    flat = tfl.Flatten(input_shape=(23, 7))(X)     # Flatten to pass into linear layers
    #d1 = tfl.Dense(30, activation='relu')(flat)
    d3 = tfl.Dense(3,activation=None)(flat)
    
    # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
    # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
    intermediate = tfl.Reshape((3,1), input_shape=(3,))(d3)
    
    probs = tfl.Cropping1D(cropping=(0,1))(intermediate)
    probs = tfl.Reshape((2,), input_shape=(2,1))(probs)
    probs = tfl.Activation('softmax')(probs)
    
    time = tfl.Cropping1D(cropping=(2,0))(intermediate)
    time = tfl.Reshape((1,), input_shape=(1,1))(time)
    
    # concatenate the probabilities and predicted_time_to_sack back into one layer
    out = tfl.Concatenate(axis=-1)([probs, time])
    
    model = Model(inputs=X, outputs=out)        # create model
    
    return model
    

In [8]:
model = createModel()

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 23, 7)]      0           []                               
                                                                                                  
 flatten (Flatten)              (None, 161)          0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 3)            486         ['flatten[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 3, 1)         0           ['dense[0][0]']                  
                                                                                              

2022-12-05 16:47:02.251834: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
LEARNING_RATE = 0.001
BETA_1 = 0.9
BETA_2 = 0.999
EPS = 1e-07

opt = optimizers.Adam(
    learning_rate=LEARNING_RATE,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

model.compile(loss = my_loss, optimizer = opt, metrics = [metrics.CategoricalAccuracy(), bce_metric, mse_metric, recall, precision])

In [10]:
# Better optimizer

lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=10000,
    decay_rate=0.9)

scheduled_opt = optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

In [11]:
NUM_EPOCHS = 50
history = model.fit(x_train[:,:,4:], y_train, epochs=NUM_EPOCHS)

Epoch 1/50
188/188 [==============================] - 2s 2ms/step - loss: 1.8754 - categorical_accuracy: 0.8920 - bce_metric: 0.2937 - mse_metric: 1.5753 - recall: 0.0129 - precision: 0.0161 
Epoch 2/50
188/188 [==============================] - 0s 1ms/step - loss: 1.2808 - categorical_accuracy: 0.9396 - bce_metric: 0.2490 - mse_metric: 1.0408 - recall: 0.0011 - precision: 0.0053
Epoch 3/50
188/188 [==============================] - 0s 2ms/step - loss: 1.1211 - categorical_accuracy: 0.9409 - bce_metric: 0.2324 - mse_metric: 0.8847 - recall: 0.0000e+00 - precision: 0.0000e+00
Epoch 4/50
188/188 [==============================] - 0s 2ms/step - loss: 1.0689 - categorical_accuracy: 0.9412 - bce_metric: 0.2271 - mse_metric: 0.8526 - recall: 0.0000e+00 - precision: 0.0000e+00
Epoch 5/50
188/188 [==============================] - 0s 2ms/step - loss: 1.0463 - categorical_accuracy: 0.9411 - bce_metric: 0.2229 - mse_metric: 0.8420 - recall: 0.0000e+00 - precision: 0.0000e+00
Epoch 6/50
188/188 [

In [29]:
train_no_sack = np.round(np.sum(y_train[:,0])/len(y_train), 3)
print(f"Percentage of training plays with no sack = {train_no_sack}")

Percentage of training plays with no sack = 0.941


In [30]:
model.predict(x_train[0:5,:,4:])

1/1 [==============================] - 0s 26ms/step


array([[ 9.9999297e-01,  7.0068982e-06, -1.2895460e+00],
       [ 9.8548108e-01,  1.4518979e-02, -8.9697844e-01],
       [ 9.9924672e-01,  7.5329869e-04, -9.7019309e-01],
       [ 9.9999297e-01,  6.9956409e-06, -1.0302030e+00],
       [ 9.9977845e-01,  2.2151478e-04, -9.6254176e-01]], dtype=float32)

In [31]:
metrics_df = pd.DataFrame(history.history)


In [32]:
metrics_df.tail()

,loss,categorical_accuracy,bce_metric,mse_metric,recall,precision
45,0.070431,0.996328,0.002741,0.067657,0.840426,0.840426
46,0.065897,0.996662,0.003180,0.062717,0.855053,0.856383
47,0.064973,0.995994,0.002808,0.062067,0.844858,0.845745
48,0.064042,0.996495,0.002801,0.061303,0.869681,0.872340
49,0.060942,0.996829,0.002549,0.058505,0.849734,0.851064


In [33]:
num_no_sack = np.round(np.sum(y_val[:,0])/len(y_val), 3)
print(f"Percentage of validation plays with no sack = {num_no_sack}")

Percentage of validation plays with no sack = 0.947


In [12]:
val_loss, cat_acc, val_bce, val_mse, val_recall, val_precision = model.evaluate(x_val[:,:,4:], y_val, verbose=2)

41/41 - 1s - loss: 0.8695 - categorical_accuracy: 0.9470 - bce_metric: 0.2117 - mse_metric: 0.6451 - recall: 0.0000e+00 - precision: 0.0000e+00 - 876ms/epoch - 21ms/step


In [13]:

print(f"val loss = {val_loss}")
print(f"categorical accuracy = {cat_acc}")
print(f"val_bce = {val_bce}")
print(f"val_mse = {val_mse}")
print(f"val_recall = {val_recall}")
print(f"val_precision = {val_precision}")

val loss = 0.8695151805877686
categorical accuracy = 0.9470404982566833
val_bce = 0.21168415248394012
val_mse = 0.6450503468513489
val_recall = 0.0
val_precision = 0.0


In [36]:
model.predict(x_val[:,:,4:])[:,0]

41/41 [==============================] - 0s 2ms/step


array([0.98357046, 0.99988365, 0.9999479 , ..., 0.9999821 , 0.99990845,
       0.9998735 ], dtype=float32)

In [37]:
# https://www.kdnuggets.com/2021/02/saving-loading-models-tensorflow.html

model_string = f"models/third_model/weights_epochs{NUM_EPOCHS}"
model.save_weights(model_string)